In [1]:
# 📦 Install if not already done
# !pip install langchain openai faiss-cpu tiktoken python-dotenv

# 1️⃣ Imports
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

import os
from dotenv import load_dotenv

# 2️⃣ Load API keys
load_dotenv(".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 3️⃣ Setup LLM
llm = ChatOpenAI(temperature=0)

# 4️⃣ Create Vector DB (Retriever)

with open("sample.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

# 🧠 Split the text into smaller chunks
from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter(separator="\n", chunk_size=300, chunk_overlap=50)
texts = splitter.split_text(text_data)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(texts, embedding)
retriever = vectorstore.as_retriever()

# 5️⃣ Create RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 6️⃣ Convert to LangChain Tool
qa_tool = Tool(
    name="LangChainRetriever",
    func=qa_chain.run,
    description="Use this to answer questions about LangChain framework, features, or its creator."
)

# 7️⃣ Setup Memory (to support conversational context)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 8️⃣ Initialize Agent with Tool + Memory
agent = initialize_agent(
    tools=[qa_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # ReAct agent
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# 9️⃣ Ask Questions (RAG-Style)
print("1️⃣ First Question")
res1 = agent.run("What is LangChain?")
print("Answer:", res1)

print("\n2️⃣ Follow-up")
res2 = agent.run("Who created it?")
print("Answer:", res2)

print("\n3️⃣ Combined Reasoning")
res3 = agent.run("Explain LangChain's use in AI workflows.")
print("Answer:", res3)


C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_18812\855904649.py:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)
C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_18812\855904649.py:31: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_18812\855904649.py:46: LangCh

1️⃣ First Question


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: LangChainRetriever
Action Input: What is LangChain?
Observation: LangChain is a framework created by Harrison Chase for building applications with Large Language Models (LLMs). It supports various features such as RAG, agents, memory, tools, and more. LangChain is commonly used in applications like chatbots, document Q&A, and AI workflow.
Thought:Do I need to use a tool? No
AI: LangChain is a framework created by Harrison Chase for building applications with Large Language Models (LLMs). It supports various features such as RAG, agents, memory, tools, and more. LangChain is commonly used in applications like chatbots, document Q&A, and AI workflow.

> Finished chain.
Answer: LangChain is a framework created by Harrison Chase for building applications with Large Language Models (LLMs). It supports various features such as RAG, agents, memory, tools, and more. LangChain is commonly us